## Final Munge Information
The result of bringing together the ETF spreadsheets and TD Mutual Fund CSVs is a complete DataFrame that is pickled in file historical_prices.pickle. 

This can be loaded as price_history. See the last few cells to shortcut the munge. 

In [1]:
import numpy as np
import pandas as pd
import datetime
import os
import pickle

## ETF Data
This section will convert the etf spreadsheets from iShares into usable data and store this information in an SQLite database. 
The etf excel spreadsheets are stored in a subdirectory called etf_data.

It is necessary to ensure each file is resaved into a modern xlsx format, as they are usually in a 2003 format.

In [2]:
# using the os 'listdir' method to return filenames in the subdirectory
path = './etf_data'
etf_files = os.listdir(path)

# set the security symbols in the same order
# as the file names in the directory
etf_symbol = ['XSB', 'XBB', 'XSP', 'XIC', 'XIN', 'XIU']
etf_asset_class = ['Fixed Income', 'Fixed Income', 
                   'Equity', 'Equity', 'Equity', 'Equity'
                  ] 

etf_geography = ['Canadian', 'Canadian', 'United States', 
                 'Canadian', 'International', 'Canadian'
                ]
# combine the two list above into a dictionary with the etf symbol as the key.
etf_details = zip(etf_files, etf_asset_class, etf_geography)
etf_dict = dict(zip(etf_symbol, etf_details))

In [3]:
# etf_symbol, etf_asset_class, etf_geography, etf_files
etf_dict['XSB']

('iShares-Core-Canadian-Short-Term-Bond-Index-ETF_fund.xlsx',
 'Fixed Income',
 'Canadian')

In [4]:
# initialize etf_df as none
etf_df = None

# loop through all the spreadsheets downloaded from iShares and create
# one dataframe with all the historical pricing and distribution
# information for dates going back to early 2000's

for symbol in etf_dict:
    file_name = etf_dict[symbol][0]
    asset_class = etf_dict[symbol][1]
    geography = etf_dict[symbol][2]

# The 'iShares-MSCI-EAFE-Index-ETF-CAD-Hedged_fund.xlsx' sheet only
# has three columns under the historical sheet, omitting the
# return-index column. Adjust the columns used when opening
# the file under read_excel so that the right columns are imported
    if file_name == 'iShares-MSCI-EAFE-Index-ETF-CAD-Hedged_fund.xlsx':
        use_columns = [0, 1, 2]
    else:
        use_columns = [0, 2, 3]

    # open spread sheet and read the data into df
    df = pd.read_excel('etf_data/' + file_name,
                       sheet_name='Historical',
                       usecols=use_columns,
                       names=['trade_date', 'nav_per_share', 'dividends'],
                       na_values="--",
                       converters={'As Of': pd.to_datetime}
                       )
    # set values in the df for symbol
    df['security'] = symbol
    
    # set values for asset class
    df['asset_class'] = asset_class

    # set values for geography
    df['asset_class'] = geography

    # create the etf_df on the first loop, else concat to exisiting etf_df.
    if etf_df is None:
        etf_df = pd.DataFrame()
        etf_df = df
    else:
        etf_df = pd.concat([etf_df, df])

# reset the index
etf_df.reset_index(inplace=True, drop=True)
etf_df.index.name = 'id'

In [5]:
# clean the NAV column to remove the NaN's
etf_df = etf_df[etf_df['nav_per_share'].notnull()]

## Pickle

In [2]:
filename = 'historical_etf.pickle'

In [11]:
# save file
outfile = open(filename, 'wb')

pickle.dump(etf_df, outfile)
outfile.close()

In [3]:
# load file
infile = open(filename, 'rb')
historical_etf = pickle.load(infile)
infile.close()

In [13]:
historical_etf.shape

(27179, 5)

## Mutual Fund Data
This section will convert the TD Mutual Fund spreadsheets into usable data and store this information in a pickle file.

The mutual fund excel spreadsheets are stored in a subdirectory called mutual_fund data.

In [50]:
# using the os 'listdir' method to return filenames in the subdirectory
path = './mutual_fund_data'
mf_files = os.listdir(path)

In [51]:
mf_files[1:5]

['TD Canadian Bond Index - e.xlsx',
 'TD Canadian Index - e.xlsx',
 'TD International Index - e.xlsx',
 'TD US Index e.xlsx']

In [52]:
# set the security symbols in the same order
# as the file names in the directory
mf_symbol = ['TD_Bond', 'TD_CDN_Equity', 'TD_INT_Equity', 'TD_US_Equity']
mf_asset_class = ['Fixed Income', 'Equity', 'Equity', 'Equity'] 

mf_geography = ['Canadian', 'Canadian', 'International', 'United States']
# combine the two list above into a dictionary with the etf symbol as the key.
mf_details = zip(mf_files[1:5], mf_asset_class, mf_geography)
mf_dict = dict(zip(mf_symbol, mf_details))

In [53]:
# etf_symbol, etf_asset_class, etf_geography, etf_files
mf_dict['TD_Bond']

('TD Canadian Bond Index - e.xlsx', 'Fixed Income', 'Canadian')

In [77]:
# initialize etf_df as none
mf_df = None

# loop through all the spreadsheets downloaded from iShares and create
# one dataframe with all the historical pricing and distribution
# information for dates going back to early 2000's

for symbol in mf_dict:
    file_name = mf_dict[symbol][0]
    asset_class = mf_dict[symbol][1]
    geography = mf_dict[symbol][2]

    # open spread sheet and read the data into df
    df = pd.read_excel('mutual_fund_data/' + file_name,
                       sheet_name=0,
                       names=['trade_date', 'nav_per_share', 'dividends'],
                       na_values="--",
                       skiprows=1, 
                       converters={'Date': pd.to_datetime}
                       )
    # set values in the df for symbol
    df['security'] = symbol
    
    # set values for asset class
    df['asset_class'] = asset_class

    # set values for geography
    df['asset_class'] = geography

    # create the etf_df on the first loop, else concat to exisiting etf_df.
    if mf_df is None:
        mf_df = pd.DataFrame()
        mf_df = df
    else:
        mf_df = pd.concat([mf_df, df])

# reset the index
mf_df.reset_index(inplace=True, drop=True)
mf_df.index.name = 'id'

In [78]:
# clean the NAV column to remove the NaN's
mf_df = mf_df[mf_df['nav_per_share'].notnull()]

In [81]:
mf_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18764 entries, 0 to 18763
Data columns (total 5 columns):
trade_date       18764 non-null datetime64[ns]
nav_per_share    18764 non-null float64
dividends        18764 non-null float64
security         18764 non-null object
asset_class      18764 non-null object
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 879.6+ KB


## Pickle

In [65]:
filename = 'historical_mf.pickle'

In [66]:
# save file
outfile = open(filename, 'wb')

pickle.dump(mf_df, outfile)
outfile.close()

NameError: name 'mf_df' is not defined

In [67]:
# load file
infile = open(filename, 'rb')
historical_mf = pickle.load(infile)
infile.close()

EOFError: Ran out of input

In [10]:
historical_mf.shape

(18764, 5)

## Join the ETF and Mutual Fund Data Frames

In [12]:
historical_mf['fund_type'] = 'Mutual Fund'

In [15]:
historical_etf['fund_type'] = 'ETF'

In [41]:
price_history_joined = pd.concat([historical_etf, historical_mf], axis=0)

In [42]:
price_history_joined.sort_values(by=['fund_type', 'security', 'trade_date'], ascending=True, inplace=True)

In [76]:
# change the type for objects to category
price_history = price_history.astype({'security': 'category', 'asset_class': 'category', 'fund_type': 'category'})

In [77]:
# replace the zero values in dividends in the mutual fund section with np.nan
price_history['dividends'] = price_history['dividends'].replace({0:np.nan})

In [78]:
price_history.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45943 entries, 9100 to 13966
Data columns (total 6 columns):
trade_date       45943 non-null datetime64[ns]
nav_per_share    45943 non-null float64
dividends        789 non-null float64
security         45943 non-null category
asset_class      45943 non-null category
fund_type        45943 non-null category
dtypes: category(3), datetime64[ns](1), float64(2)
memory usage: 1.5 MB


## Pickle the final dataframe

In [71]:
filename = 'historical_prices.pickle'

In [79]:
# save file
outfile = open(filename, 'wb')

pickle.dump(price_history, outfile)
outfile.close()

In [80]:
# load file
infile = open(filename, 'rb')
price_history = pickle.load(infile)
infile.close()

In [81]:
price_history.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45943 entries, 9100 to 13966
Data columns (total 6 columns):
trade_date       45943 non-null datetime64[ns]
nav_per_share    45943 non-null float64
dividends        789 non-null float64
security         45943 non-null category
asset_class      45943 non-null category
fund_type        45943 non-null category
dtypes: category(3), datetime64[ns](1), float64(2)
memory usage: 1.5 MB
